## Modelo 3

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

In [22]:
df = pd.read_csv('dataset6.csv')
df.head()


,temperatura,sensacion_termica,humedad,velocidad_viento,total_alquileres,temporada_1,temporada_2,temporada_3,temporada_4,anio_0,...,dia_semana_3.0,dia_semana_4.0,dia_semana_5.0,dia_semana_6.0,dia_trabajo_0,dia_trabajo_1,clima_1,clima_2,clima_3,clima_4
0,0.24,0.2879,0.81,0.0,16.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1,0.22,0.2727,0.80,0.0,40.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0.22,0.2727,0.80,0.0,32.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.24,0.2879,0.75,0.0,13.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
4,0.24,0.2879,0.75,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0


In [23]:
# Separar características (X) y etiqueta (y)
X = df.drop(columns=["total_alquileres"])
y = df["total_alquileres"]

In [24]:
y_log = np.log1p(y)

In [25]:
# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y_log, test_size=0.2, random_state=42)

In [26]:
numeric_cols = ['temperatura', 'sensacion_termica', 'humedad', 'velocidad_viento']
ohe_columns = [col for col in X.columns if col not in numeric_cols]

In [27]:
# Suponiendo que 'numeric_cols' son las columnas numéricas originales
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),  # Escalar numéricas
        ('cat', 'passthrough', ohe_columns)       # No tocar las dummies
    ])

In [28]:
X_train_scaled = preprocessor.fit_transform(X_train)
X_test_scaled = preprocessor.fit_transform(X_test)

In [29]:
# Escalar características para modelos sensibles a la escala
'''
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)\
'''

'\nscaler = StandardScaler()\nX_train_scaled = scaler.fit_transform(X_train)\nX_test_scaled = scaler.transform(X_test)'

In [30]:
def evaluar_modelo(modelo, X_train, X_test, y_train, y_test):
    modelo.fit(X_train, y_train)

    # Predicciones en entrenamiento
    pred_train = modelo.predict(X_train)
    rmse_train = np.sqrt(mean_squared_error(y_train, pred_train))
    r2_train = r2_score(y_train, pred_train)

    # Predicciones en validación
    pred_test = modelo.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, pred_test))
    r2_test = r2_score(y_test, pred_test)

    return rmse_train, r2_train, rmse_test, r2_test, modelo

In [31]:
def modelo_lineal(X_train, X_test, y_train, y_test):
    lr = LinearRegression()
    return evaluar_modelo(lr, X_train, X_test, y_train, y_test)

In [32]:
def modelo_lasso(X_train, X_test, y_train, y_test, alphas=[0.01, 0.1, 1.0]):
    grid = GridSearchCV(Lasso(max_iter=10000), {'alpha': alphas}, cv=3, scoring='neg_root_mean_squared_error')
    grid.fit(X_train, y_train)
    return evaluar_modelo(grid.best_estimator_, X_train, X_test, y_train, y_test)

In [33]:
def modelo_elasticnet(X_train, X_test, y_train, y_test, params=None):
    if params is None:
        params = {'alpha': [0.01, 0.1, 1.0], 'l1_ratio': [0.1, 0.5, 0.9]}
    grid = GridSearchCV(ElasticNet(max_iter=10000), params, cv=3, scoring='neg_root_mean_squared_error')
    grid.fit(X_train, y_train)
    return evaluar_modelo(grid.best_estimator_, X_train, X_test, y_train, y_test)

In [34]:
def modelo_ridge(X_train, X_test, y_train, y_test, alphas=[0.1, 1.0, 10]):
    grid = GridSearchCV(Ridge(), {'alpha': alphas}, cv=3, scoring='neg_root_mean_squared_error')
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_
    return evaluar_modelo(best_model, X_train, X_test, y_train, y_test)

In [35]:
def modelo_gradient_boosting(X_train, X_test, y_train, y_test, n_iter=10):
    param_dist = {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 10]
    }
    gb = RandomizedSearchCV(GradientBoostingRegressor(), param_dist, n_iter=n_iter, cv=3,
                            scoring='neg_root_mean_squared_error', n_jobs=-1, random_state=42)
    gb.fit(X_train, y_train)
    return evaluar_modelo(gb.best_estimator_, X_train, X_test, y_train, y_test)

In [36]:
def modelo_random_forest(X_train, X_test, y_train, y_test, n_iter=10):
    param_dist = {
        'n_estimators': [100, 200],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }
    rf = RandomizedSearchCV(RandomForestRegressor(random_state=42), param_dist, n_iter=n_iter, cv=3,
                            scoring='neg_root_mean_squared_error', n_jobs=-1, random_state=42)
    rf.fit(X_train, y_train)
    return evaluar_modelo(rf.best_estimator_, X_train, X_test, y_train, y_test)

In [37]:
def modelo_gradient_boosting_hibrido(X_train, X_test, y_train, y_test,
                                     param_dist, n_iter=20, cv=3):
    # Paso 1: RandomizedSearchCV (exploración)
    random_search = RandomizedSearchCV(
        GradientBoostingRegressor(),
        param_distributions=param_dist,
        n_iter=n_iter,
        cv=cv,
        scoring='neg_root_mean_squared_error',
        n_jobs=-1,
        random_state=42
    )
    random_search.fit(X_train, y_train)

    best_params = random_search.best_params_

    # Paso 2: GridSearchCV (explotación)
    param_grid = {
        'n_estimators': list(set([
            max(10, best_params['n_estimators'] - 50),  # evitando valores negativos
            best_params['n_estimators'],
            best_params['n_estimators'] + 50
        ])),
        'max_depth': list(set([
            max(1, best_params['max_depth'] - 1),
            best_params['max_depth'],
            best_params['max_depth'] + 1
        ])),
        'learning_rate': sorted(list(set([
            round(best_params['learning_rate'] * 0.5, 3),
            best_params['learning_rate'],
            round(best_params['learning_rate'] * 1.5, 3)
        ])))
    }

    grid_search = GridSearchCV(
        GradientBoostingRegressor(subsample=best_params['subsample']),
        param_grid=param_grid,
        cv=cv,
        scoring='neg_root_mean_squared_error',
        n_jobs=-1
    )
    grid_search.fit(X_train, y_train)

    return evaluar_modelo(grid_search.best_estimator_, X_train, X_test, y_train, y_test)

In [38]:
def modelo_random_forest_hibrido(X_train, X_test, y_train, y_test,
                                 param_dist, n_iter=20, cv=3):
    # Paso 1: RandomizedSearchCV
    random_search = RandomizedSearchCV(
        RandomForestRegressor(random_state=42),
        param_distributions=param_dist,
        n_iter=n_iter,
        cv=cv,
        scoring='neg_root_mean_squared_error',
        n_jobs=-1,
        random_state=42
    )
    random_search.fit(X_train, y_train)
    best_params = random_search.best_params_

    # Paso 2: GridSearchCV
    param_grid = {
        'n_estimators': sorted(set([
            max(10, best_params['n_estimators'] - 50),
            best_params['n_estimators'],
            best_params['n_estimators'] + 50
        ])),
        'max_depth': [best_params['max_depth']] if best_params['max_depth'] is None else sorted(set([
            max(1, best_params['max_depth'] - 5),
            best_params['max_depth'],
            best_params['max_depth'] + 5
        ])),
        'min_samples_split': sorted(set([
            max(2, best_params['min_samples_split'] - 1),
            best_params['min_samples_split'],
            best_params['min_samples_split'] + 1
        ])),
        'min_samples_leaf': sorted(set([
            max(1, best_params['min_samples_leaf'] - 1),
            best_params['min_samples_leaf'],
            best_params['min_samples_leaf'] + 1
        ]))
    }

    grid_search = GridSearchCV(
        RandomForestRegressor(random_state=42),
        param_grid=param_grid,
        cv=cv,
        scoring='neg_root_mean_squared_error',
        n_jobs=-1
    )
    grid_search.fit(X_train, y_train)

    # Evaluación con tu función
    return evaluar_modelo(grid_search.best_estimator_, X_train, X_test, y_train, y_test)


In [39]:
# Llamada de ejemplo con métricas de entrenamiento y validación
rmse_lr_train, r2_lr_train, rmse_lr_test, r2_lr_test, modelo_lr = modelo_lineal(X_train_scaled, X_test_scaled, y_train, y_test)
rmse_ridge_train, r2_ridge_train, rmse_ridge_test, r2_ridge_test, modelo_ridge = modelo_ridge(X_train_scaled, X_test_scaled, y_train, y_test)
rmse_lasso_train, r2_lasso_train, rmse_lasso_test, r2_lasso_test, modelo_lasso = modelo_lasso(X_train_scaled, X_test_scaled, y_train, y_test)

# Impresión de resultados
print(f"Lineal (Train)  -> RMSE: {rmse_lr_train:.2f}, R²: {r2_lr_train:.2f}")
print(f"Lineal (Test)   -> RMSE: {rmse_lr_test:.2f}, R²: {r2_lr_test:.2f}")

print(f"Ridge (Train)   -> RMSE: {rmse_ridge_train:.2f}, R²: {r2_ridge_train:.2f}")
print(f"Ridge (Test)    -> RMSE: {rmse_ridge_test:.2f}, R²: {r2_ridge_test:.2f}")

print(f"Lasso (Train)   -> RMSE: {rmse_lasso_train:.2f}, R²: {r2_lasso_train:.2f}")
print(f"Lasso (Test)    -> RMSE: {rmse_lasso_test:.2f}, R²: {r2_lasso_test:.2f}")

Lineal (Train)  -> RMSE: 0.59, R²: 0.83
Lineal (Test)   -> RMSE: 0.60, R²: 0.82
Ridge (Train)   -> RMSE: 0.59, R²: 0.83
Ridge (Test)    -> RMSE: 0.60, R²: 0.82
Lasso (Train)   -> RMSE: 0.64, R²: 0.80
Lasso (Test)    -> RMSE: 0.64, R²: 0.79


In [40]:
rmse_gb_train, r2_gb_train, rmse_gb_test, r2_gb_test, modelo_gb = modelo_gradient_boosting(X_train, X_test, y_train, y_test)

In [41]:
print(f"Gradient Boosting (Train) -> RMSE: {rmse_gb_train:.2f}, R²: {r2_gb_train:.2f}")
print(f"Gradient Boosting (Test)  -> RMSE: {rmse_gb_test:.2f}, R²: {r2_gb_test:.2f}")

Gradient Boosting (Train) -> RMSE: 0.08, R²: 1.00
Gradient Boosting (Test)  -> RMSE: 0.33, R²: 0.94


In [42]:
rmse_rf_train, r2_rf_train, rmse_rf_test, r2_rf_test, modelo_rf = modelo_random_forest(X_train, X_test, y_train, y_test)

In [43]:
print(f"Random Forest (Train) -> RMSE: {rmse_rf_train:.2f}, R²: {r2_rf_train:.2f}")
print(f"Random Forest (Test)  -> RMSE: {rmse_rf_test:.2f}, R²: {r2_rf_test:.2f}")

Random Forest (Train) -> RMSE: 0.12, R²: 0.99
Random Forest (Test)  -> RMSE: 0.34, R²: 0.94


In [44]:
param_dist_gb = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0]
}

rmse_gb_train, r2_gb_train, rmse_gb_test, r2_gb_test, modelo_gb = modelo_gradient_boosting_hibrido(X_train, X_test, y_train, y_test, param_dist_gb)

In [45]:
print(f"Gradient Boosting (Train)  -> RMSE: {rmse_gb_train:.2f}, R²: {r2_gb_train:.2f}")
print(f"Gradient Boosting (Test)   -> RMSE: {rmse_gb_test:.2f}, R²: {r2_gb_test:.2f}")
print("Mejores hiperparámetros Gradient Boosting:", modelo_gb.get_params())

Gradient Boosting (Train)  -> RMSE: 0.24, R²: 0.97
Gradient Boosting (Test)   -> RMSE: 0.33, R²: 0.95
Mejores hiperparámetros Gradient Boosting: {'alpha': 0.9, 'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.2, 'loss': 'squared_error', 'max_depth': 4, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 300, 'n_iter_no_change': None, 'random_state': None, 'subsample': 0.6, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}


Quizas un random forest hibrido

Vamos a hacer que X no tenga escalamiento ya que sus variables estan todas normalizadas de 0 a 1

In [46]:
param_dist_rf = {
    'n_estimators': [100, 150, 200, 250],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rmse_rf_train, r2_rf_train, rmse_rf_test, r2_rf_test, modelo_rf = modelo_random_forest_hibrido(
    X_train, X_test, y_train, y_test, param_dist_rf
)

print(f"Random Forest Híbrido (Train) -> RMSE: {rmse_rf_train:.2f}, R²: {r2_rf_train:.2f}")
print(f"Random Forest Híbrido (Test)  -> RMSE: {rmse_rf_test:.2f}, R²: {r2_rf_test:.2f}")
print("Mejores hiperparámetros Random Forest:", modelo_rf.get_params())

KeyboardInterrupt: 